<b>

ISYS613 | Course Project | Fall 2020
--------|----------------|----------

</b>

## Project Overview
The objective of this project is to afford you the opportunity to demonstrate the
skills acquired over the course of the semester.  Specifically, you will be constructing
an analytics-ready dataset using data from several disparate, but related data sources.

Your dataset is to be a pandas dataframe suitable for answering the proof of concept
questions located at the end of this document.

### The Scenario
You are the proprietor of a small tour company.  You are interested in launching a new app
that will allow users to undertake self-guided tours of noteworthy authors. To launch your app
you need to assemble the authors' location information along with some other facts that you
believe will be of interest to your customers.

## Data Sources
You will be collecting and assembling data from several different sources and formats.

### Source 1: Initial Author Data
The initial list of author names is to be web-scraped from a popular "famous quotes" website.
There are 50 unique authors available from this site.  Begin your exploration at the base
URL and determine your own strategy to find and capture the required author data.

Depending on your approach, you may encounter the same author multiple times.
In the end, your list of authors should be duplicate free.

```
Base Scrape URL: https://quotes.toscrape.com
```
#### Tasks
1. Beginning at the above URL, you are to use a web scraping technique to collect
the following author information:
* Full Name
* Date of birth
* Birth location
2. Create a pandas dataframe from (or using) the relevant data

In [3]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin
from requests.exceptions import HTTPError
import pandas as pd
import numpy as np
from pprint import pprint
import csv


# SOURCE 1 SOLUTION

SCRAPE_URL =  'https://quotes.toscrape.com'
my_lst=[]
a_lst =[]
a_info= []
data = {
    'name':[],
    'dob':[],
    'pob':[],   
}

def simple_get(url, *args, **kwargs):
    try:
        resp = requests.get(url, *args, **kwargs)
        # If the response was successful, no Exception will be raised
        resp.raise_for_status()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        raise http_err
    except Exception as err:
        print(f'Other error occurred: {err}')
        raise err

    return resp

def soup_data(resp):
    html = resp.content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def get_about_us(url):
    page = requests.get(url)
    
    if page.status_code == requests.codes.ok:
        html = page.content
        my_soup = BeautifulSoup(html, 'html.parser')
        for content in my_soup.find_all("a", href=re.compile("/author/")):
            a_url = content.get('href')
            a_url = SCRAPE_URL + a_url
            if a_url in my_lst:
                continue;
            else:
                my_lst.append(a_url)
                a_resp = simple_get(a_url)
                a_soup = soup_data(a_resp)
                name = a_soup.find('div', class_='author-details').text.strip()
                dob = a_soup.find('span', class_='author-born-date').text.strip()
                location = a_soup.find('span', class_='author-born-location').text.strip()[3:]
                data['name'].append(name)
                data['dob'].append(dob)
                data['pob'].append(location)       
        
        next_page = my_soup.find("li", class_="next")
        if next_page is not None:
            next_page_text = my_soup.find("li", class_="next").text
            next_page_text = next_page_text.strip()[0:4]
            if next_page_text == 'Next':
                next_url = my_soup.find("li", class_="next").find("a")["href"]
                full_url = SCRAPE_URL + next_url
                get_about_us(full_url)
    else:
        print("invalid url")
            
    return data

author_data = get_about_us(SCRAPE_URL)
author_df = pd.DataFrame(author_data)
author_df


,name,dob,pob
0,Albert Einstein,"March 14, 1879","Ulm, Germany"
1,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit..."
2,Jane Austen,"December 16, 1775","Steventon Rectory, Hampshire, The United Kingdom"
3,Marilyn Monroe,"June 01, 1926",The United States
4,André Gide,"November 22, 1869","Paris, France"
5,Thomas A. Edison,"February 11, 1847","Milan, Ohio, The United States"
6,Eleanor Roosevelt,"October 11, 1884",The United States
7,Steve Martin,"August 14, 1945","Waco, Texas, The United States"
8,Bob Marley,"February 06, 1945","Nine Mile, Saint Ann, Jamaica"
9,Dr. Seuss,"March 02, 1904","Springfield, MA, The United States"


### Source 2: Author Key Data
For each of the 50 authors previously identified, you are to merge a key and gender value available
from a CSV file with the author data from *Source 1*.

```
CSV File: *author_key_file.csv*
```
#### Tasks
1. The author names are unique in both
data sources and thus may be used to associate the *key* and *gender* attribute values with the author.
3. Once you have completed the merge, convert the *key* column to be the dataframe's row index.

In [4]:
CSV_FILE = 'author_key_file.csv'

ak_df= pd.read_csv(CSV_FILE, encoding='utf-8', delimiter=',', index_col=0)
merged_df = pd.merge(author_df,ak_df, on='name', how='inner' )
indexed_df = merged_df.set_index('key')
indexed_df

,name,dob,pob,gender
key,,,,
QWxiZXJ0,Albert Einstein,"March 14, 1879","Ulm, Germany",M
Si1LLVJv,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",F
SmFuZS1B,Jane Austen,"December 16, 1775","Steventon Rectory, Hampshire, The United Kingdom",F
TWFyaWx5,Marilyn Monroe,"June 01, 1926",The United States,F
QW5kcmUt,André Gide,"November 22, 1869","Paris, France",M
VGhvbWFz,Thomas A. Edison,"February 11, 1847","Milan, Ohio, The United States",M
RWxlYW5v,Eleanor Roosevelt,"October 11, 1884",The United States,F
U3RldmUt,Steve Martin,"August 14, 1945","Waco, Texas, The United States",M
Qm9iLU1h,Bob Marley,"February 06, 1945","Nine Mile, Saint Ann, Jamaica",M


### Source 3: Author Location Data
This direct download XML data source contains the location information that will be used
to direct app users to the historical sites associated with each author.  The *id*
tag can be used to match the previously assembled author data (ie, the *key* attribute) with
the author location data.

You are to retain all of the location attributes (excluding *id* - which will be a
duplicate of the existing *key* attribute) from this data source to extend the
previously collected author data.

```
Direct Download URL: https://www.drivehq.com/file/DFPublishFile.aspx/FileID7657515244/Keycqacws4cypvo/author_location_data.xml
Data Format: XML
Encoding: UTF-8
```
#### Tasks
1. Ingest the XML data into a pandas dataframe
2. Use the pandas dataframe merge method to join the
two datasets and keep only the data related to our list of 50 authors.
3. *Source 3* contains location data for many public figures in addition
to the 50 authors in which we are currently interested. Be sure to restrict your
results to our 50 authors.

In [5]:
import xml.etree.ElementTree as et
LOC_URL = 'https://www.drivehq.com/file/DFPublishFile.aspx/FileID7657515244/Keycqacws4cypvo/author_location_data.xml'

response = requests.get(LOC_URL)
with open('author_location.xml', 'wb') as file:
    file.write(response.content)

xtree = et.parse('author_location.xml')
xroot = xtree.getroot()

col = ['key','price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 
           'waterfront', 'grade', 'yr_built', 'lat', 'long']
rows =[]

for node in xroot:
    author_id = node.find('id').text if node is not None else None
    price = float(node.find('price').text) if node is not None else None
    bedrooms = int(node.find('bedrooms').text) if node is not None else None
    bathrooms = float(node.find('bathrooms').text) if node is not None else None
    sqft_living = int(node.find('sqft_living').text) if node is not None else None
    sqft_lot = int(node.find('sqft_lot').text) if node is not None else None
    floors = node.find('floors').text if node is not None else None
    waterfront = node.find('waterfront').text if node is not None else None
    grade = node.find('grade').text if node is not None else None
    yr_built = node.find('yr_built').text if node is not None else None
    lat = float(node.find('lat').text) if node is not None else None
    long = float(node.find('long').text) if node is not None else None
    rows.append({'key':author_id, 'price':price, 'bedrooms':bedrooms, 'bathrooms':bathrooms, 
                'sqft_living':sqft_living,'sqft_lot':sqft_lot,'floors':floors,'waterfront':waterfront,
                'grade':grade,'yr_built':yr_built,'lat':lat, 'long':long})

xml_df = pd.DataFrame(rows, columns=col)
#xml_df
    
merged_xml_df = pd.merge(indexed_df, xml_df, on='key', how='inner') 
author_info_df= merged_xml_df.set_index('key')
author_info_df


,name,dob,pob,gender,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,grade,yr_built,lat,long
key,,,,,,,,,,,,,,,
QWxiZXJ0,Albert Einstein,"March 14, 1879","Ulm, Germany",M,699999.0,3,0.75,1240,4000,1,0,7,1968,40.7123,-94.0385
Si1LLVJv,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",F,415000.0,3,2.50,1060,1536,2,0,8,2000,38.8778,-77.0672
SmFuZS1B,Jane Austen,"December 16, 1775","Steventon Rectory, Hampshire, The United Kingdom",F,300000.0,3,2.50,2540,5050,2,0,7,2006,33.9784,-117.0405
TWFyaWx5,Marilyn Monroe,"June 01, 1926",The United States,F,2480000.0,4,5.00,5310,16909,1,0,12,1992,42.8802,-78.8432
QW5kcmUt,André Gide,"November 22, 1869","Paris, France",M,425000.0,3,1.75,1530,9800,1,0,8,1958,37.6119,-82.7187
VGhvbWFz,Thomas A. Edison,"February 11, 1847","Milan, Ohio, The United States",M,179900.0,2,1.00,680,6400,1,0,6,1943,47.7890,-117.0142
RWxlYW5v,Eleanor Roosevelt,"October 11, 1884",The United States,F,420000.0,3,1.75,1770,6000,1,0,7,1952,34.6356,-102.7123
U3RldmUt,Steve Martin,"August 14, 1945","Waco, Texas, The United States",M,761000.0,3,3.50,2050,2020,2,0,8,2006,42.0902,-72.0532
Qm9iLU1h,Bob Marley,"February 06, 1945","Nine Mile, Saint Ann, Jamaica",M,307150.0,3,1.50,1480,6752,1,0,7,1959,32.7916,-96.7526


### Source 4: Zip Code API
You may have observed that the *Source 3* data does not include the city and zip code information for
the property locations for our 50 authors.  Your task is the use the latitude and longitude
values from the *Source 3* data to locate the city and zip code information and incorporate
that information into your dataframe.

The USPS maintains data related to all US zip codes and their centroid latitude and longitude.  The
*Source 3* data contains the latitude and longitude information (the lat and long tags) for the
author locations. Your task
is to use the *Source 3* latitude and longitude values as an argument to the zip code api in order to
to augment your dataframe with the city name and zip code of the authors' locations.

The ZIP code API contains ZIP codes for the
continental United States, Alaska, Hawaii, Puerto Rico, and American Samoa.
The API provides data in JSON format values for for ZIP code, city, latitude, longitude,
timezone (offset from GMT).

The relevant zip code API URL and parameters are shown in the table below.

API Information | Value
---------------------|------
API URL | https://public.opendatasoft.com/api/records/1.0/search/. [Docs](https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/api/)
dataset (parameter) | us-zip-code-latitude-and-longitude
rows (parameter) | 100
q (parameter) | TBD by You. Hint: use the #distance function. If used, distance argument must by at least 10 kilometers. [Docs](https://help.opendatasoft.com/platform/en/exploring_catalog_and_datasets/03_searching_the_data/search.html#full-text-search)
geofilter.distance (parameter) | TBD by You. If used, distance value must by at least 10 kilometers. [Docs](https://help.opendatasoft.com/platform/en/exploring_catalog_and_datasets/03_searching_the_data/search.html#geo-filtering)
Notes: | use only one of either the *q* or *geofilter.distance* parameters. Do not use both.  The choice
is yours.

#### Tasks
The long and lat values from *Source 3* are the exact longitude and latitude values for a
single property location.  As such, it is unlikely that you will find an exact match from the
zip code API using these values. Instead, you must:

1. round the lat and long values to 2 decimal place
2. invoke the API requesting all zip code related data within 10 kilometers from the
rounded lat and long values. (Refer to the *q* or *geofilder.distance* parameter documentation listed above.)
3. If the returned JSON results indicate more that one candidate record has been returned, you
must determine which record's longitude and latitude are the **closest** to the *Source 3*
long and lat values.
<br/><br/>There are several ways to accomplish this. For example, you could calculate the distance between the
each result's longitude,latitude values and the the long, lat values from *Source 3*. The
distance between the values can be calculated using the Pythagorean Theorem as follows: <br/>
&radic;( (lon<sub>1</sub>-lon<sub>2</sub>)<sup>2</sup> + (lat<sub>1</sub>-lat<sub>2</sub>)<sup>2</sup> )
 <br/>
 <br/>
 Another approach invoves a carefull reading the documentation and examination of the returned results.
4. Use the city and zip code that are associated with the result having the
 minimum distance from (closest to) the *Source 3* long, lat values.

In [7]:
import math
# SOURCE 4 SOLUTION
API_URL = 'https://public.opendatasoft.com/api/records/1.0/search'

decimals = 2
min_dist=1000
key_counter = 0
api_data = {
    'key':[],
    'city':[],
    'zip':[],   
}
author_info_df['lat'] = author_info_df['lat'].apply(lambda x: round(x, decimals))
author_info_df['long'] = author_info_df['long'].apply(lambda x: round(x, decimals))
geo_lst = author_info_df.loc[:, 'lat':'long']
print(geo_lst)
geo_lst.reset_index(drop=False, inplace=True)
for x in range(len(geo_lst)):
    y = geo_lst.loc[x,'lat']
    z = geo_lst.loc[x,'long']
    auth_key = geo_lst.loc[x,'key']
    parameters = dict(dataset='us-zip-code-latitude-and-longitude', rows=100, geofilterdistance=(y,z, 10000)) 
    api_resp = simple_get(API_URL, params=parameters, timeout=5)
    py_dict = api_resp.json()
    geo_data = py_dict['records']
    for locations in geo_data:
        api_lat = float(locations['fields']['latitude'])
        api_long = float(locations['fields']['longitude'])
        api_lat = round(api_lat,decimals)
        api_long = round(api_long, decimals)
        distance = math.sqrt((y-api_lat)**2 +(z-api_long)**2)
        distance = round(distance,decimals)
        if distance < min_dist:
            min_dist = distance
            zip_code =locations['fields']['zip']
            city = locations['fields']['city']  
        else:
            continue
    key_counter = key_counter + 1
    if key_counter > 0:
        api_data['key'].append(auth_key)
        api_data['city'].append(city)
        api_data['zip'].append(zip_code)
        min_dist= 1000 

api_df = pd.DataFrame(api_data)
merge_api_df = pd.merge(author_info_df, api_df, on='key', how='inner')
merge_api_df  

            lat    long
key                    
QWxiZXJ0  40.71  -94.04
Si1LLVJv  38.88  -77.07
SmFuZS1B  33.98 -117.04
TWFyaWx5  42.88  -78.84
QW5kcmUt  37.61  -82.72
VGhvbWFz  47.79 -117.01
RWxlYW5v  34.64 -102.71
U3RldmUt  42.09  -72.05
Qm9iLU1h  32.79  -96.75
RHItU2V1  42.67  -73.79
RG91Z2xh  26.74  -80.11
RWxpZS1X  37.34  -85.33
RnJpZWRy  42.88  -78.84
TWFyay1U  40.05  -84.11
QWxsZW4t  45.53 -102.79
UGFibG8t  37.98  -89.04
UmFscGgt  35.51  -80.82
TW90aGVy  37.43  -96.92
R2Fycmlz  44.58  -71.17
SmltLUhl  32.35  -86.27
Q3hhcmxl  38.03  -78.47
V2lsbGlh  48.77  -94.94
Sm9yZ2Ut  34.05 -117.23
R4Vvcmdl  37.65  -75.66
R5Vvcmdl  40.83  -74.27
Qy1TLUxl  31.47  -92.77
TWFydGlu  38.90  -77.02
SmFtZXMt  40.28  -76.87
SGFydWtp  43.73  -97.59
QWxleGFu  40.43  -89.30
U3RlcGhl  21.31 -157.84
RXJuZXN0  40.12  -90.56
SGVsZW4t  41.68  -73.80
R2Vvcmdl  44.86  -72.58
Q2hhcmxl  30.71  -86.04
U3V6YW5u  32.51  -92.14
Si1SLVIt  39.96  -75.15
QWxmcmVk  37.53  -82.62
VGVycnkt  38.86 -104.76
Si1ELVNh  44.26 

,key,name,dob,pob,gender,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,grade,yr_built,lat,long,city,zip
0,QWxiZXJ0,Albert Einstein,"March 14, 1879","Ulm, Germany",M,699999.0,3,0.75,1240,4000,1,0,7,1968,40.71,-94.04,Leeton,64761
1,Si1LLVJv,J.K. Rowling,"July 31, 1965","Yate, South Gloucestershire, England, The Unit...",F,415000.0,3,2.50,1060,1536,2,0,8,2000,38.88,-77.07,Washington,20594
2,SmFuZS1B,Jane Austen,"December 16, 1775","Steventon Rectory, Hampshire, The United Kingdom",F,300000.0,3,2.50,2540,5050,2,0,7,2006,33.98,-117.04,Moreno Valley,92555
3,TWFyaWx5,Marilyn Monroe,"June 01, 1926",The United States,F,2480000.0,4,5.00,5310,16909,1,0,12,1992,42.88,-78.84,Buffalo,14219
4,QW5kcmUt,André Gide,"November 22, 1869","Paris, France",M,425000.0,3,1.75,1530,9800,1,0,8,1958,37.61,-82.72,East Stone Gap,24246
5,VGhvbWFz,Thomas A. Edison,"February 11, 1847","Milan, Ohio, The United States",M,179900.0,2,1.00,680,6400,1,0,6,1943,47.79,-117.01,Spokane,99217
6,RWxlYW5v,Eleanor Roosevelt,"October 11, 1884",The United States,F,420000.0,3,1.75,1770,6000,1,0,7,1952,34.64,-102.71,Sugar City,81076
7,U3RldmUt,Steve Martin,"August 14, 1945","Waco, Texas, The United States",M,761000.0,3,3.50,2050,2020,2,0,8,2006,42.09,-72.05,Arlington Heights,02475
8,Qm9iLU1h,Bob Marley,"February 06, 1945","Nine Mile, Saint Ann, Jamaica",M,307150.0,3,1.50,1480,6752,1,0,7,1959,32.79,-96.75,Rockwall,75087
9,RHItU2V1,Dr. Seuss,"March 02, 1904","Springfield, MA, The United States",M,550000.0,3,3.50,2490,3582,2,0,8,2005,42.67,-73.79,Albany,12230


## Proof of Concept
Use the dataframe that you constructed above to answer the following three questions.

### Question 1
What is the mean and standard deviation of living space (sqft_living) by number of bedrooms? Use the pandas
dataframe.agg() method to calculate these statistics in a single step.

### Question 2
How many authors, by gender, live on lots that exceed the mean lot size (sqft_lot) by more
than 1 standard deviation.

### Question 3
Examine the documentation for the pandas.cut() class method
[Docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html)

Use the cut class method to convert the *price* attribute into to
3 approximately equal sized bins. Assign each bin specific labels of 'low', 'medium', 'high'.
How many authors are there in each price category?

In [10]:
# PROOF OF CONCEPT SOLUTION
#Question 1
sqft_df_agg = merge_api_df.groupby('bedrooms').agg({'sqft_living':[np.mean, np.std]})
print(f' Mean and Std Dev of living space\n {sqft_df_agg}')

 Mean and Std Dev of living space
           sqft_living             
                 mean          std
bedrooms                          
2          972.000000   270.314631
3         1824.230769   504.271142
4         2454.777778  1066.275130
5         4180.000000          NaN


In [68]:
#Question 2

lot_mean = merge_api_df['sqft_lot'].mean()
lot_std = merge_api_df['sqft_lot'].std()

gender_df = merge_api_df.loc[(merge_api_df['sqft_lot'] > (lot_mean + lot_std))]
gender_df = gender_df.groupby('gender').size()
gender_df


gender
F    1
M    2
dtype: int64

In [18]:
#Question 3
#min_val = merge_api_df['price'].min()
#mean_val = merge_api_df['price'].mean()
#max_val = merge_api_df['price'].max()
#cut_bins = [0, min_val,mean_val,max_val]
bin_labels = ['low', 'mid', 'high']
df_price = pd.cut(merge_api_df['price'], bins=3, labels=bin_labels).value_counts()
df_price 

low     48
high     1
mid      1
Name: price, dtype: int64